## **Importation des packages et des données**

In [5]:
import pandas as pd
import pyproj
import numpy as np
import plotly.graph_objects as go
import openchord as ocd
import matplotlib as plt
import matplotlib.colors as mcolors

In [ ]:
df_2016_2018 = pd.read_csv('data/data_2016_2018.csv', low_memory=False)
df_2019 = pd.read_csv('data/data_2019.csv', low_memory=False)
df_2020 = pd.read_csv('data/data_2020.csv', low_memory=False)
df_2021 = pd.read_csv('data/data_2021.csv', low_memory=False)
df_2022 = pd.read_csv('data/data_2022.csv', low_memory=False)
df_2023 = pd.read_csv('data/data_2023.csv', low_memory=False)
df_capture = pd.read_csv('data/df_cmr.csv', low_memory = False)

### Nettoyage des données

In [ ]:
df_combined = pd.concat([df_2016_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_capture], ignore_index=True)
df_combined.head()

In [ ]:
def convert_l93_to_latlon(x, y):
    transformer = pyproj.Transformer.from_crs("epsg:2154", "epsg:4326", always_xy=True)
    lon, lat = transformer.transform(x, y)
    return lat, lon

In [ ]:
df_combined['LAT_WGS'], df_combined['LONG_WGS'] = convert_l93_to_latlon(df_combined['LONG_L93'], df_combined['LAT_L93'])

In [ ]:
df_clean = df_combined[['DATE', 'ANNEE', 'COMMUNE', 'LIEU_DIT', 'PRECISION_MILIEU', 'DEPARTEMENT', 'LONG_L93', 'LAT_L93', 'LONG_WGS', 'LAT_WGS', 'HEURE', 'CODE_ESP', 'MASSE', 'AB', 'SEXE', 'ACTION', 'ID_PIT', 'NUM_PIT']]

In [ ]:
df_clean.to_csv('data/df_clean.csv', index=False)

In [ ]:
df_clean.head()

In [ ]:
mask_not_null = df_clean['HEURE'].notna()

# Vérifier si toutes les valeurs non-nulles contiennent le séparateur ":"
contains_separator = df_clean.loc[mask_not_null, 'HEURE'].str.contains(':', na=False)

# Identifier les mauvais séparateurs (par exemple, autres que ":")
mauvais_separateurs = df_clean.loc[mask_not_null & ~contains_separator, 'HEURE']

# Afficher les valeurs avec mauvais séparateurs
print("Valeurs avec mauvais séparateurs :")
print(mauvais_separateurs)

mauvais_separateurs.to_csv('data/df_error_HEURE.csv', index = False)

In [ ]:
# Supprimer les guillemets de la colonne HEURE
df_clean = pd.read_csv('data/df_clean.csv', low_memory = False)
df_clean['HEURE'] = df_clean['HEURE'].astype(str).str.replace('"', '')

# Remplacer les virgules par des points dans la colonne HEURE
df_clean['HEURE'] = df_clean['HEURE'].str.replace(',', '.')

# Définir une fonction pour convertir une heure décimale en HH:MM:SS
def convert_decimal_hour_to_hms(decimal_hour):
    decimal_hour = float(decimal_hour)
    hours = int(decimal_hour)
    minutes = int((decimal_hour - hours) * 60)
    seconds = int(((decimal_hour - hours) * 60 - minutes) * 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}" if not np.isnan(decimal_hour) else np.nan

# Appliquer la conversion aux valeurs nécessitant une conversion (qui contiennent un point)
mask_convert = df_clean['HEURE'].str.contains('\.') & ~df_clean['HEURE'].isnull()

df_clean.loc[mask_convert, 'HEURE'] = df_clean.loc[mask_convert, 'HEURE'].apply(lambda x: convert_decimal_hour_to_hms(x))

df_clean.to_csv('data/df_clean.csv', index = False)


In [ ]:
df_clean.head()

## **Importation des données propres**

In [2]:
def load_data_antenna():
    df_antenna = pd.read_csv(r'C:\Users\Gamy\Documents\GitHub\dashboard-taipy-bat-CMR\data\df_clean.csv', engine="pyarrow", dtype_backend="pyarrow")
    df_antenna['DATE'] = df_antenna['DATE'].astype('datetime64[ns]')
    df_antenna['HEURE'] = pd.to_datetime(df_antenna['HEURE'], format='%H:%M:%S').dt.time
    df_antenna['ANNEE'] = df_antenna['ANNEE'].astype('int')
    df_antenna['COMMUNE'] = df_antenna['COMMUNE'].astype('str')
    df_antenna['LIEU_DIT'] = df_antenna['LIEU_DIT'].astype('str')
    df_antenna['PRECISION_MILIEU'] = df_antenna['PRECISION_MILIEU'].astype('str')
    df_antenna['DEPARTEMENT'] = df_antenna['DEPARTEMENT'].astype('str')
    df_antenna['CODE_ESP'] = df_antenna['CODE_ESP'].astype('str')
    df_antenna['MASSE'] = df_antenna['MASSE'].astype('str')
    df_antenna['AB'] = df_antenna['AB'].astype('str')
    df_antenna['SEXE'] = df_antenna['SEXE'].astype('str')
    df_antenna['ACTION'] = df_antenna['ACTION'].astype('str')
    df_antenna['ID_PIT'] = df_antenna['ID_PIT'].astype('str')
    df_antenna['NUM_PIT'] = df_antenna['NUM_PIT'].astype('str')
    df_antenna['LONG_L93'] = df_antenna['LONG_L93'].astype('float')
    df_antenna['LAT_L93'] = df_antenna['LAT_L93'].astype('float')
    df_antenna['LONG_WGS'] = df_antenna['LONG_WGS'].astype('float')
    df_antenna['LAT_WGS'] = df_antenna['LAT_WGS'].astype('float')
    return df_antenna

df_antenna = load_data_antenna()

## **Chord diagram**

In [ ]:
def create_transition_matrix(df, remove_self_loops=True, reduce_self_loops=False, reduction_factor=0.5):
    # Trier par individu et date pour suivre les transitions
    df = df.sort_values(by=['NUM_PIT', 'DATE'])

    # Créer une colonne pour le lieu précédent
    df['LIEU_PRECEDENT'] = df.groupby('NUM_PIT')['LIEU_DIT'].shift()

    # Filtrer pour obtenir seulement les transitions valides (non nulles)
    df_transitions = df.dropna(subset=['LIEU_PRECEDENT'])

    # Compter les transitions de chaque lieu vers un autre
    transition_counts = df_transitions.groupby(['LIEU_PRECEDENT', 'LIEU_DIT']).size().reset_index(name='count')

    # Retirer les transitions où source == target si demandé
    if remove_self_loops:
        transition_counts = transition_counts[transition_counts['LIEU_PRECEDENT'] != transition_counts['LIEU_DIT']]

    # Réduire le poids des transitions de recontrôle si demandé
    if reduce_self_loops:
        transition_counts.loc[transition_counts['LIEU_PRECEDENT'] == transition_counts['LIEU_DIT'], 'count'] *= reduction_factor

    # Construire une matrice de transition
    lieux = sorted(set(df['LIEU_DIT'].unique()) | set(df['LIEU_PRECEDENT'].dropna().unique()))
    transition_matrix = pd.DataFrame(0, index=lieux, columns=lieux)

    for _, row in transition_counts.iterrows():
        transition_matrix.at[row['LIEU_PRECEDENT'], row['LIEU_DIT']] = row['count']

    # Retourner la matrice de transition et les labels
    return transition_matrix, lieux

# Créer la matrice de transition en retirant les recontrôles
transition_matrix, labels = create_transition_matrix(df_antenna, remove_self_loops=True)

In [ ]:
transition_matrix.head()

In [ ]:
fig = ocd.Chord(transition_matrix, labels)
fig.radius = 1000
fig.plot_area = {"x": -1300, "y": -1300, "w": 2600, "h": 2600}
fig.save_svg("plots/chord_diagram_cmr.svg")
fig.show()

In [ ]:
df_antenna_filtered = df_antenna[(df_antenna['DEPARTEMENT'] == "79") | (df_antenna['DEPARTEMENT'] == "17")]
transition_matrix, labels = create_transition_matrix(df_antenna_filtered, remove_self_loops=True)
fig = ocd.Chord(transition_matrix, labels)
fig.radius = 1000
fig.plot_area = {"x": -1300, "y": -1300, "w": 2600, "h": 2600}
fig.save_svg("plots/chord_diagram_cmr.svg")
fig.show()

## **Carte des trajectoires**

In [ ]:
def create_trajectories_map_old(df, width=1200, height=1000):
    if df.empty:
        fig = go.Figure(go.Scattermapbox())
        fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=6, mapbox_center={"lat": 46.493889, "lon": 2.602778}, height=height, width=width)
        return fig

    # Définir les centres de la carte basés sur les moyennes des latitudes et longitudes
    center_lat, center_lon = df['LAT_WGS'].mean(), df['LONG_WGS'].mean()

    fig = go.Figure()

    # Extraction des liaisons uniques entre sites
    unique_connections = df.sort_values('DATE').drop_duplicates(subset=['LAT_WGS', 'LONG_WGS'], keep='first')

    # Tracer les liaisons
    last_point = None
    for _, row in unique_connections.iterrows():
        if last_point is not None:
            fig.add_trace(go.Scattermapbox(
                lat=[last_point['LAT_WGS'], row['LAT_WGS']],
                lon=[last_point['LONG_WGS'], row['LONG_WGS']],
                mode='lines',
                line=dict(color='violet', width=2),
                showlegend=False
            ))
        last_point = row

    # Ajouter les marqueurs pour les sites, avec noms dans la légende
    sites = df.drop_duplicates(subset=['LAT_WGS', 'LONG_WGS', 'LIEU_DIT'])
    for _, site in sites.iterrows():
        fig.add_trace(go.Scattermapbox(
            lat=[site['LAT_WGS']],
            lon=[site['LONG_WGS']],
            mode='markers+text',
            marker=go.scattermapbox.Marker(size=9, color='blue'),
            textfont=dict(color='black'),  # Spécifiez la couleur du texte en noir
            text=site['LIEU_DIT'],
            textposition="bottom right",
            name=site['LIEU_DIT'],  # Utiliser le lieu dit comme nom dans la légende
            showlegend=False
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=6,
        mapbox_center={"lat": center_lat, "lon": center_lon},
        height=height,
        width=width
    )
    return fig

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def create_trajectories_map_from_matrix(df, transition_matrix, lieux, width=1200, height=1000):
    # Récupérer les coordonnées associées à chaque LIEU_DIT
    coords = df.drop_duplicates(subset=['LIEU_DIT'])[['LIEU_DIT', 'LAT_WGS', 'LONG_WGS']].set_index('LIEU_DIT')

    fig = go.Figure()

    # Transformer la matrice en table de connexions
    transition_table = transition_matrix.stack().reset_index()
    transition_table.columns = ['source', 'target', 'count']

    # Filtrer pour ne garder que les connexions avec au moins une occurrence
    transition_table = transition_table[transition_table['count'] > 0]

    # Obtenir les limites pour normaliser les valeurs de count
    min_count = transition_table['count'].min()
    max_count = transition_table['count'].max()

    # Créer une colormap de bleu à rouge
    cmap = plt.cm.get_cmap('coolwarm')

    # Tracer les connexions
    for _, row in transition_table.iterrows():
        lat1, lon1 = coords.loc[row['source'], ['LAT_WGS', 'LONG_WGS']]
        lat2, lon2 = coords.loc[row['target'], ['LAT_WGS', 'LONG_WGS']]
        
        # Calculer la couleur basée sur la fréquence
        norm = (row['count'] - min_count) / (max_count - min_count)  # Normaliser entre 0 et 1
        color = mcolors.to_hex(cmap(norm))  # Obtenir la couleur hexadécimale

        fig.add_trace(go.Scattermapbox(
            lat=[lat1, lat2],
            lon=[lon1, lon2],
            mode='lines',
            line=dict(color=color, width=min(5, row['count'] / 10)),  # La largeur de la ligne est proportionnelle au nombre d'occurrences
            showlegend=False
        ))

    # Ajouter les marqueurs pour les sites
    for lieu, coord in coords.iterrows():
        fig.add_trace(go.Scattermapbox(
            lat=[coord['LAT_WGS']],
            lon=[coord['LONG_WGS']],
            mode='markers+text',
            marker=go.scattermapbox.Marker(size=7, color='blue'),
            textfont=dict(color='black'),
            text=lieu,
            textposition="bottom right",
            name=lieu,
            showlegend=False
        ))

    # Définir le centre de la carte
    center_lat, center_lon = df['LAT_WGS'].mean(), df['LONG_WGS'].mean()

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=6,
        mapbox_center={"lat": center_lat, "lon": center_lon},
        height=height,
        width=width
    )

    return fig

In [ ]:
def create_trajectories_map_full(df, width=1200, height=1000):
    if df.empty:
        fig = go.Figure(go.Scattermapbox())
        fig.update_layout(
            mapbox_style="open-street-map",
            mapbox_zoom=6,
            mapbox_center={"lat": 46.493889, "lon": 2.602778},
            height=height,
            width=width
        )
        return fig

    # Définir les centres de la carte basés sur les moyennes des latitudes et longitudes
    center_lat, center_lon = df['LAT_WGS'].mean(), df['LONG_WGS'].mean()

    fig = go.Figure()

    # Trier par individu et date
    df_sorted = df.sort_values(by=['NUM_PIT', 'DATE'])

    # Utiliser un set pour garder une trace des connexions déjà tracées
    traced_connections = set()

    # Tracer les liaisons pour chaque individu
    for num_pit, group in df_sorted.groupby('NUM_PIT'):
        last_point = None
        for _, row in group.iterrows():
            if last_point is not None:
                # Créer une clé unique pour chaque connexion (ordre des points ne compte pas)
                connection_key = frozenset(((last_point['LAT_WGS'], last_point['LONG_WGS']), (row['LAT_WGS'], row['LONG_WGS'])))
                
                # Ajouter la connexion seulement si elle n'a pas déjà été tracée
                if connection_key not in traced_connections:
                    fig.add_trace(go.Scattermapbox(
                        lat=[last_point['LAT_WGS'], row['LAT_WGS']],
                        lon=[last_point['LONG_WGS'], row['LONG_WGS']],
                        mode='lines',
                        line=dict(color='violet', width=2),
                        showlegend=False
                    ))
                    # Marquer la connexion comme tracée
                    traced_connections.add(connection_key)

            last_point = row

    # Ajouter les marqueurs pour les sites, avec noms dans la légende
    sites = df.drop_duplicates(subset=['LAT_WGS', 'LONG_WGS', 'LIEU_DIT'])
    for _, site in sites.iterrows():
        fig.add_trace(go.Scattermapbox(
            lat=[site['LAT_WGS']],
            lon=[site['LONG_WGS']],
            mode='markers+text',
            marker=go.scattermapbox.Marker(size=9, color='blue'),
            textfont=dict(color='black'),  # Spécifiez la couleur du texte en noir
            text=site['LIEU_DIT'],
            textposition="bottom right",
            name=site['LIEU_DIT'],  # Utiliser le lieu dit comme nom dans la légende
            showlegend=False
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=6,
        mapbox_center={"lat": center_lat, "lon": center_lon},
        height=height,
        width=width
    )
    return fig

In [3]:
def create_transition_matrix(df, remove_self_loops=True, reduce_self_loops=False, reduction_factor=0.5):
    # Trier par individu et date pour suivre les transitions
    df = df.sort_values(by=['NUM_PIT', 'DATE'])

    # Créer une colonne pour le lieu précédent
    df['LIEU_PRECEDENT'] = df.groupby('NUM_PIT')['LIEU_DIT'].shift()

    # Filtrer pour obtenir seulement les transitions valides (non nulles)
    df_transitions = df.dropna(subset=['LIEU_PRECEDENT'])

    # Compter les transitions de chaque lieu vers un autre
    transition_counts = df_transitions.groupby(['LIEU_PRECEDENT', 'LIEU_DIT']).size().reset_index(name='count')

    # Retirer les transitions où source == target si demandé
    if remove_self_loops:
        transition_counts = transition_counts[transition_counts['LIEU_PRECEDENT'] != transition_counts['LIEU_DIT']]

    # Réduire le poids des transitions de recontrôle si demandé
    if reduce_self_loops:
        transition_counts.loc[transition_counts['LIEU_PRECEDENT'] == transition_counts['LIEU_DIT'], 'count'] *= reduction_factor

    # Construire une matrice de transition
    lieux = sorted(set(df['LIEU_DIT'].unique()) | set(df['LIEU_PRECEDENT'].dropna().unique()))
    transition_matrix = pd.DataFrame(0, index=lieux, columns=lieux)

    for _, row in transition_counts.iterrows():
        transition_matrix.at[row['LIEU_PRECEDENT'], row['LIEU_DIT']] = row['count']

    return transition_matrix, lieux

def process_transition_matrix(transition_matrix):
    # Transformer la matrice en table de connexions
    transition_table = transition_matrix.stack().reset_index()
    transition_table.columns = ['source', 'target', 'count']

    # Filtrer pour ne garder que les connexions avec au moins une occurrence
    transition_table = transition_table[transition_table['count'] > 0]

    return transition_table

def create_trajectories_map(df, transition_table, width=1200, height=1000):
    # Récupérer les coordonnées associées à chaque LIEU_DIT
    coords = df.drop_duplicates(subset=['LIEU_DIT'])[['LIEU_DIT', 'LAT_WGS', 'LONG_WGS']].set_index('LIEU_DIT')

    fig = go.Figure()

    # Obtenir les limites pour normaliser les valeurs de count
    min_count = transition_table['count'].min()
    max_count = transition_table['count'].max()

    # Créer une colormap personnalisée de vert à rouge
    cmap = mcolors.LinearSegmentedColormap.from_list("GreenOrangeRed", ["green", "orange", "red"])

    # Tracer les connexions
    for _, row in transition_table.iterrows():
        lat1, lon1 = coords.loc[row['source'], ['LAT_WGS', 'LONG_WGS']]
        lat2, lon2 = coords.loc[row['target'], ['LAT_WGS', 'LONG_WGS']]
        
        # Calculer la couleur basée sur la fréquence
        norm = (row['count'] - min_count) / (max_count - min_count)  # Normaliser entre 0 et 1
        color = mcolors.to_hex(cmap(norm))  # Obtenir la couleur hexadécimale

        fig.add_trace(go.Scattermapbox(
            lat=[lat1, lat2],
            lon=[lon1, lon2],
            mode='lines',
            line=dict(color=color, width=1),  # Largeur fixe pour toutes les lignes
            showlegend=False
        ))

    # Ajouter les marqueurs pour les sites
    for lieu, coord in coords.iterrows():
        fig.add_trace(go.Scattermapbox(
            lat=[coord['LAT_WGS']],
            lon=[coord['LONG_WGS']],
            mode='markers+text',
            marker=go.scattermapbox.Marker(size=7, color='blue'),
            textfont=dict(color='black'),
            text=lieu,
            textposition="bottom right",
            name=lieu,
            showlegend=False
        ))

    # Définir le centre de la carte
    center_lat, center_lon = df['LAT_WGS'].mean(), df['LONG_WGS'].mean()

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=6,
        mapbox_center={"lat": center_lat, "lon": center_lon},
        height=height,
        width=width
    )

    return fig

In [7]:
transition_matrix, lieux = create_transition_matrix(df_antenna)

# Traitement de la matrice pour obtenir un tableau des connexions
transition_table = process_transition_matrix(transition_matrix)


In [31]:
transition_table.head()

,source,target,count
1672,Ancien tunnel de la Peytivie,Les Tours de Merle - Tour Fulcon,5
5417,Barrage de l'Aigle,Centrale hydroélectrique de Claredent,6
5623,Barrage de l'Aigle,Les Tours de Merle - Tour Fulcon,12
6036,Barrières,Les Dames,5
6062,Barrières,Les Tours de Merle - Tour Fulcon,5


In [35]:
transition_table[(transition_table['source'] == "Cave Billard") & (transition_table['target'] == "Château d'Argenton")].head()

,source,target,count
47582,Cave Billard,Château d'Argenton,30


In [9]:
transition_table.describe()

,count
count,2147.000000
mean,10.166744
std,60.382758
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,1370.000000


In [34]:
transition_table = process_transition_matrix(transition_matrix)
transition_table = transition_table[transition_table['count'] > 4]

len_table = len(transition_table)
print(f"Number of transitions: {len_table}")

min_count = transition_table['count'].min()
print("Valeur minimale de 'count':", min_count)

max_count = transition_table['count'].max()
print("Valeur maximale de 'count':", max_count)

Number of transitions: 439
Valeur minimale de 'count': 5
Valeur maximale de 'count': 1370
